In [2]:
from owslib.wfs import WebFeatureService
wfs11 = WebFeatureService(url='https://www.imis.bfs.de/ogc/opendata/ows', version='1.1.0')
wfs11.identification.title

'BfS Download'

In [5]:
typeName='opendata:odlinfo_timeseries_odl_24h'

In [6]:
# getting dataset1
import json
from owslib.fes import *
from owslib.etree import etree
import xml.dom.minidom 

filter = PropertyIsLike(propertyname='kenn', literal='091861251',wildCard='*')
filterxml = etree.tostring(filter.toXML()).decode("utf-8")
response = wfs11.getfeature(typename=typeName,filter = filterxml,outputFormat='application%2Fjson',maxfeatures=10)
# convert IO-byte to bytes
bytesD=bytes(response.read())
# convert to json
data = json.loads(bytesD)
print(data)

{'type': 'FeatureCollection', 'features': [], 'totalFeatures': 0, 'numberMatched': 0, 'numberReturned': 0, 'timeStamp': '2022-10-14T09:50:20.985Z', 'crs': None}


In [ ]:
#multiple filter1

import json
from owslib.fes import *
from owslib.etree import etree
import xml.dom.minidom 

filter1 = PropertyIsGreaterThanOrEqualTo(propertyname='start', literal='2022-09-01')
filter2 = PropertyIsLike(propertyname='id', literal='DEZ0005',wildCard='*')
#filters3= PropertyIsLike(propertyname='local_authority', literal='Cham',wildCard='*')
filters=[filter1,filter2]

filterxml = etree.tostring(And(operations=filters).toXML()).decode("utf-8")


response = wfs11.getfeature(typename=typeName,filter = filterxml,outputFormat='application%2Fjson',maxfeatures=100)
# convert IO-byte to bytes
bytesD=bytes(response.read())
# convert to json
data = json.loads(bytesD)

In [ ]:
#save json1 to show in qgis
import json
with open(typeName + '.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
import pandas as pd
#using pandas for serializing
df= pd.json_normalize(data,"features")
df

In [20]:
# Import Library

import matplotlib.pyplot as plt


# Create dataframe

dataframe = pd.DataFrame(data,columns=[df['properties.start'], df['properties.value']])
 
# Changing the datatype

dataframe[df['properties.start']] = dataframe[df['properties.start']].astype("datetime64")
 
# Setting the Date as index

dataframe = dataframe.set_index(df['properties.start'])


# Plot

plt.plot(dataframe[df['properties.value']], marker='o')

# Labelling 

plt.xlabel(df['properties.start'])
plt.ylabel("Temp in Faherenheit")
plt.title("Pandas Time Series Plot")

# Display

plt.show()

ValueError: Length mismatch: Expected 0 rows, received array of length 100